In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np

Using TensorFlow backend.


## Call elmo from tensorflow hub

In [3]:
url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/bilm/CNN/b_cnn_3:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/CNN/b_cnn_3
INFO:tensorflow:Initialize variable module/bilm/CNN/W_cnn_5:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/CNN/W_cnn_5
INFO:tensorflow:Initialize variable module/bilm/CNN_high_1/b_transform:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/CNN_high_1/b_transform
INFO:tensorflow:Initialize variable module/aggregation/scaling:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/scaling
INFO:tensorflow:Initialize variable module/bilm/CNN/b_cnn_6:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/CNN/b_cnn_

In [4]:
data = pd.read_csv('data/spam.csv', encoding='latin-1')

In [5]:
y = list(data['v1'])
x = list(data['v2'])

In [6]:
le = preprocessing.LabelEncoder()
le.fit(y)

LabelEncoder()

In [7]:
def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

In [8]:
test = encode(le, ['ham', 'spam', 'ham', 'ham'])

In [9]:
untest = decode(le, test)

/home/hulk/.virtualenvs/py3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [10]:
test

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [11]:
untest

array(['ham', 'spam', 'ham', 'ham'], dtype='<U4')

In [12]:
x_enc = x
y_enc = encode(le, y)

In [13]:
x_train = np.asarray(x_enc[:5000])
y_train = np.asarray(y_enc[:5000])

In [14]:
x_test = np.asarray(x_enc[5000:])
y_test = np.asarray(y_enc[5000:])

In [15]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

In [16]:
def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [17]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(2, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=1, batch_size=32)
    model.save_weights('./elmo-model.h5')

Epoch 1/1
5000/5000 [==============================] - 70s 14ms/step - loss: 0.1915 - acc: 0.9516


In [19]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./elmo-model.h5')  
    predicts = model.predict(x_test, batch_size=32)

In [20]:
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

/home/hulk/.virtualenvs/py3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/hulk/.virtualenvs/py3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [21]:
from sklearn import metrics

In [22]:
print(metrics.confusion_matrix(y_test, y_preds))

[[487  11]
 [ 32  42]]


In [23]:
print(metrics.classification_report(y_test, y_preds))

             precision    recall  f1-score   support

        ham       0.94      0.98      0.96       498
       spam       0.79      0.57      0.66        74

avg / total       0.92      0.92      0.92       572

